### Парсинг телефонов с Авито

In [12]:
import sys
sys.path.append('../..')


In [13]:
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from tqdm import tqdm
from kafka import KafkaProducer
import psycopg2
import json
from selenium_stealth import stealth
from webdriver_manager.chrome import ChromeDriverManager
import random

from src.parser.models import Product, Seller
from src.parser.parser import parse_avito_page
from src.parser.tools import get_ad_urls

### Константы

In [ ]:
URL_TEMPLATE = "https://www.avito.ru/sankt-peterburg/telefony/mobilnye_telefony/apple-ASgBAgICAkS0wA3OqzmwwQ2I_Dc?p="
NUM_PAGES = 10

producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

def send_phone(data):
    producer.send("phone_listings", value=data)
    producer.flush()

def send_seller(data):
    producer.send("seller_listings", value=data)
    producer.flush()


# списки существующих ключей для того, чтобы лишний раз 
# не добавлять объявление/ продавца в  таблицу
conn = psycopg2.connect("dbname=mydatabase user=myuser password=mypassword host=localhost port=5432")
cursor = conn.cursor()

cursor.execute("SELECT link FROM product")
product_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT seller_id FROM seller")
seller_ids = [row[0] for row in cursor.fetchall()]

cursor.close()
conn.close()

### Запуск сервиса

In [15]:
service = Service(ChromeDriverManager().install())

options = webdriver.ChromeOptions()
# options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

### Максимальный номер сохраненной страницы  

In [16]:
start_page_number = 10
end_page_number = start_page_number + NUM_PAGES

print(f"Начнем парсинг со страницы номер: {start_page_number} (вкл)")
print(f"Закончим парсинг на странице номер: {end_page_number} (не вкл)")

Начнем парсинг со страницы номер: 10 (вкл)
Закончим парсинг на странице номер: 20 (не вкл)


### Парсинг

In [ ]:
for page_num in range(start_page_number, end_page_number):
    print(f"Начали парсинг страницы # {page_num}")
    try:
        # Загрузка страницы с объявлениями
        url = URL_TEMPLATE + str(page_num)
        driver.get(url)
        time.sleep(random.randint(2, 5))

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
  
        # Извлекаем ссылки на объявления на текущей странице
        links = get_ad_urls(soup)
        for link in tqdm(links):
            if link in product_ids:
                continue
            try:
                # Переход на страницу объявления
                driver.get(link)
                time.sleep(random.randint(2, 5))

                # Парсим данные на странице объявления (название, цена, фото, описание и т.д.)
                ad_data, seller_data, done_deals_data = parse_avito_page(driver=driver) # <- словарик 
                ad_data['link'] = link

                # валидация используя Pydantic
                product = Product.model_validate(ad_data)
                seller = Seller.model_validate(seller_data)

                if ad_data['link'] in product_ids:
                    continue
                
                product_ids.append(link)
                send_phone(product.model_dump())
                # проверка на отсутствие продавца в таблице
                if seller_data['seller_id'] not in seller_ids:
                    seller_ids.append(seller_data['seller_id'])
                    send_seller(seller.model_dump())

                    done_deals_list = [Product.model_validate(deal) for deal in done_deals_data]
                    for deal in done_deals_list:
                        send_phone(deal.model_dump())
            except TimeoutException:
                print(f"Ошибка: объявление {link} не загрузилось, пропускаем...")
                continue
            except Exception as e:
                print(f"Произошла ошибка при обработке объявления {link}: {e}")
                continue
    except TimeoutException:
        print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
        continue
    except Exception as e:
        print(f"Произошла ошибка на странице {page_num}: {e}")
        continue


Начали парсинг страницы # 10


 44%|████▍     | 22/50 [07:39<11:38, 24.94s/it]

Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_8_plus_64_gb_7317081199?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJOT3JkdE1rakNqSWxLcVc1Ijt9jr8Cez8AAAA: 1 validation error for Product
seller_id
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


 62%|██████▏   | 31/50 [10:54<07:38, 24.11s/it]

Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_11_128_gb_4673260468?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJOT3JkdE1rakNqSWxLcVc1Ijt9jr8Cez8AAAA: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: Unable to receive message from renderer
  (Session info: chrome=135.0.7049.114)
Stacktrace:
0   chromedriver                        0x00000001005f6e10 cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x00000001005ef0ac cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x00000001001368d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100120d9c cxxbridge1$string$len + 4136
4   chromedriver                        0x0000000100120b14 cxxbridge1$string$len + 3488
5   chromedriver                        0x000000010011fb34 chromedriver + 195380
6   chromedriver                        

100%|██████████| 50/50 [10:55<00:00, 13.12s/it]

Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_13_128_gb_7240206455?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiJOT3JkdE1rakNqSWxLcVc1Ijt9jr8Cez8AAAA: Message: invalid session id
Stacktrace:
0   chromedriver                        0x00000001005f6e10 cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x00000001005ef0ac cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x000000010013674c cxxbridge1$string$len + 92632
3   chromedriver                        0x0000000100170ec8 cxxbridge1$string$len + 332116
4   chromedriver                        0x000000010019929c cxxbridge1$string$len + 496936
5   chromedriver                        0x00000001001985a4 cxxbridge1$string$len + 493616
6   chromedriver                        0x00000001001069d4 chromedriver + 92628
7   chromedriver                        0x00000001005bbc98 cxxbridge1$str$ptr + 2575000
8   chromedriver

In [18]:
driver.quit()